# Preparation of W States

This notebook walks through example code to prepare the generic W-state using the methods outlined in LINK.
This wavepacket preparation step is the first key step of probing the energy dependent transport using wavepacket dynamics.

This notebook is divided into three sections.
1. Unitary Circuit Demonstration
2. MCM-FF-1 Circuit Demonstration
3. MCM-FF-2 Circuit Demonstration

## Unitary Circuit Demonstration

To prepare states of the form $\sum_{n} c_{n} |2^n\rangle$, the authors of Ref. [1](https://arxiv.org/abs/1807.05572) present a method to prepare W-states with logarithmically scaling circuit depth.
The approach requires no mid-circuit measurements.
The circuit is structured to have a cascading series of CX and RY gates.
The wavepacket coefficients are tuned using $\theta$ Ry rotation angles.
The derivation of these angles is expressed as the solution to a linear system of equations, which we solve below using the `sympy` package.

<img src="figures/unitary_circuit.png" width="300" height="220" />

In [5]:
# Import packages
import numpy as np

# Import linear system of equation solvers
from sympy import sin, cos, Symbol, Array, Eq, lambdify, Float
from sympy.solvers import solve, nsolve

# Import tools for circuit construction
from pytket.circuit import Circuit
import numpy as np
from pytket.circuit.display import render_circuit_jupyter as draw

In [6]:
# Set the parameters and coefficients
N = 56	# Number of qubits
c = np.zeros(N) + 1/np.sqrt(N)	# Uniform coefficients

tol = 1e-6	# Solution tolerance when solving linear system of equations

In [7]:
# First, solve for the rotations given |c|. 
# The imaginary part of c can be applied via Rz rotations at the end of the circuit.
abs_c = np.abs(c)

# Solve linear system of equations for Ry coefficients
# Define the variables in sympy
ry_theta = [Symbol(f't{i}') for i in range(0, N-1)]

# Construct \Theta
big_theta = np.ones((N, N), dtype=object)

for n in range(N):
	for j in range(0, min(2**n, N - 2**n)):
		big_theta[j][2**n + j] = cos(ry_theta[2**n + j - 1] / 2)
		big_theta[2**n + j][2**n + j] = sin(ry_theta[2**n + j - 1] / 2)

for n_p in range(1, N):
	for n in range(n_p, N):
		for j in range(0, min(2**n, N - 2**n) - 1 + 1):
			for jp in range(j, -1, -2):
				big_theta[2**n + j][jp] = big_theta[2**(n - n_p) + j - 1][2**(n - n_p) + jp - 1]
				big_theta[2**n + j][2**(n - n_p) + jp] = big_theta[j][2**(n - n_p) + jp]

equation_sys = []

# Obtain list of equations
for i in range(len(big_theta)):
	row = big_theta[i]
	equation = 1
	for exp in row:
		equation *= exp

	equation = Eq(equation, Float(abs_c[i]))
	equation_sys.append(equation)

guess = np.ones(N-1) # Define an arbitrary guess
thetas = nsolve(equation_sys, ry_theta, guess, tol=tol)
thetas = np.array(thetas, dtype=float).T[0] # Reshape to fit expected type

In [8]:
# Construct circuit in Pytket
qc = Circuit(N, name='log')

cr = qc.add_c_register('cr', N)

qc.X(0)

# Add cascading series of CX and Ry gates
for i in range(int(np.ceil(np.log2(N)))):
	for j in range(2**i):
		if j + 2**i < N:
			qc.CRy(thetas[j + 2**i - 1] / np.pi, j, j + 2**i)
			qc.CX(j + 2**i, j)

# add Rz rotations - adjusted by pi (see Pytket documentation)
for i in range(N):
	qc.Rz(np.angle(c)[i] / np.pi, i)

for i in range(N):
	qc.Measure(i, cr[i])

In [9]:
# Draw the unitary circuit
draw(qc)

## MCM-FF-1 Circuit Demonstration

We reference a fused approach to state preparation.
The MCM-FF-1 method first prepares the state on each half of the qubit register, and then uses a single Bell measurement to ``fuse" these states together into the larger W state.
This method is expected to be successful half the time.

<img src="figures/mcmff1_circuit.png" width="300" height="220" />

In [1]:
# Import packages
import numpy as np
from tools import fused_log_subcircuit_pytket

# Import tools for circuit construction
from pytket.circuit import Circuit
import numpy as np
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.circuit.logic_exp import (
	if_bit
)

In [2]:
N = 56	# Number of qubits
c = 1/np.sqrt(N)

In [3]:
qc = Circuit(name='mcmff1 subcircuit')
qr = qc.add_q_register('q', N)
ar = qc.add_c_register('ar', 2)
cr = qc.add_c_register('cr', N)
	
# define indices for subcircuits
mid = int(N / 2)

# calculate the subindices
b = np.zeros(int(N / 2))
a = np.zeros(int(np.ceil(N / 2))) # in the case of odd values

b[:mid - 2] = c
b[mid - 2] = np.sqrt(2/N)
b[mid - 1] = np.sqrt(1 / 2)

a[0] = np.sqrt(1 / 2)
a[1] = np.sqrt(2/N)
a[2:] = c

# get the subcircuits
lower_subcircuit = fused_log_subcircuit_pytket(mid, b)
upper_subcircuit = fused_log_subcircuit_pytket(mid, a)

# get the indices of the subcircuits
lower_slice = [None] * mid
upper_slice = [None] * mid

for i in range(mid):
	lower_slice[i] = qr[i]
	upper_slice[i] = qr[mid + i]

# append subcircuit
qc.add_circuit(lower_subcircuit, lower_slice)
qc.add_circuit(upper_subcircuit, upper_slice)

# obtain MCM
qc.CX(qr[mid - 1], qr[mid])
qc.H(qr[mid - 1])
qc.Measure(qr[mid - 1], ar[0]).Reset(qr[mid - 1])
qc.Measure(qr[mid], ar[1]).Reset(qr[mid])

qc.add_clexpr_from_logicexp(ar[0] & ar[1], [ar[0]])

# add conditional Z gates
for i in range(mid + 1, N):
	qc.Z(qr[i], condition=if_bit(ar[0]))

# calculate rotation angles
theta = np.arctan(1) * 2

# prepare state psi_2
qc.CRy(theta / np.pi, qr[mid - 2], qr[mid - 1])
qc.CX(qr[mid - 1], qr[mid - 2])

qc.CRy(theta / np.pi, qr[mid + 1], qr[mid])
qc.CX(qr[mid], qr[mid + 1])

for i in range(N):
	qc.Measure(qr[i], cr[i])

In [4]:
draw(qc)

## MCM-FF-2 Circuit Demonstration

This is the constant depth ancilla-free method for preparing W states presented in Ref. [2](https://arxiv.org/abs/2505.03111).
The method takes a user-defined input parameter $\delta$ that correlates with the shot success rate and prepared state fidelity.
The number of mid-circuit measurements made on this circuit is significantly higher than that of the unitary and MCM-FF-1 methods, but MCM-FF-2 is expected to be ideal for asymptotically large systems.

<img src="figures/mcmff2_circuit.png" width="300" height="230" />

In [11]:
# Import packages
import numpy as np

# Import tools for circuit construction
from pytket.circuit import Circuit
import numpy as np
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.circuit.logic_exp import (
	if_bit
)

In [12]:
# Define parameters
N = 56				# Number of qubits
delta = 0.2			# User-defined parameter correlates with success rate
c = 1 / np.sqrt(N)	# preparing the W-state with uniform distribution

phi = 0.0
theta = np.arcsin(np.sqrt(delta) * np.sqrt(2 * c**2)) * 2

In [13]:
# Initialize circuit
qc = Circuit(name='mcm_ff')

# Determine number of X and Z readout registers
num_xr = int(N / 4)
num_zr = int(N / 2) - 1

# Add registers
qr = qc.add_q_register('qr', N)
xr = qc.add_c_register('xr', num_xr)
zr = qc.add_c_register('zr', num_zr)
ar = qc.add_c_register('ar', 1)

# Initialize final measurement register
cr = qc.add_c_register('cr', N)

for i in range(1, N, 2):
	qc.Ry(theta / np.pi, qr[i])

# -- Prepare state 1 --
qc.H(qr[0])

# controlled prod Z gate
for i in range(2, N, 4):
	qc.H(qr[i])

# First layer of cnot gates
for i in range(2, N - 2, 4):
	qc.CX(qr[i], qr[i + 2])

# Second layer of cnot gates
for i in range(0, N - 6, 4):
	qc.CX(qr[i], qr[i + 2])

for i in range(num_xr):
	qc.Measure(qr[2 + i * 4], xr[i]).Reset(qr[2 + i * 4])

	# Back propagate parities
	for j in range(i):
		qc.add_clexpr_from_logicexp(xr[i] ^ xr[j], [xr[j]])

# Apply X only on odd parity gates, i.e. XOR = 1
for i in range(num_xr - 1):
	qc.X(qr[(i + 1) * 4], condition=if_bit(xr[i]))

# Third layer of CNOT gates
for i in range(0, N - 2, 4):
	qc.CX(qr[i], qr[i + 2])

for i in range(0, N - 1, 2):
	qc.CRz(1, qr[i], qr[i+1])
	
# Apply H gates
for i in range(2, N, 2):
	qc.H(qr[i])
	
# Calculate parity of Z gate
for i in range(num_zr):
	qc.Measure(qr[2 + 2*i], zr[i])
	if i == 0:
		qc.add_clexpr_from_logicexp(bool(int(N / 4) % 2) ^ zr[i], [zr[i]])
	else:
		qc.add_clexpr_from_logicexp(zr[i] ^ zr[i - 1], [zr[i]])

# Apply Z gate only for odd parity
qc.Z(qr[0], condition=if_bit(zr[num_zr - 1]))

qc.H(qr[0])
qc.Measure(qr[0], ar[0])

# Reset all measured bits (a bit different from X parity measurement approach)
for i in range(0, N, 2):
	qc.Reset(qr[i])
	
# -- Prepare state 2 --
ctrl_theta = np.arctan(1) * 2

for i in range(0, N - 1, 2):
	qc.CRy(ctrl_theta / np.pi, qr[i + 1], qr[i], condition=if_bit(ar[0]))
	qc.CX(qr[i], qr[i + 1], condition=if_bit(ar[0]))

# Measure and save to specific register
for i in range(N):
	qc.Measure(qr[i], cr[i])

In [15]:
draw(qc)

# References
[1] Cruz, D., Fournier, R., Gremion, F., Jeannerot, A., Komagata, K., Tosic, T., ... & Javerzac‐Galy, C. (2019). Efficient quantum algorithms for GHZ and W states, and implementation on the IBM quantum computer. *Advanced Quantum Technologies*, 2(5-6), 1900015.

[2] Farrell, R. C., Zemlevskiy, N. A., Illa, M., & Preskill, J. (2025). Digital quantum simulations of scattering in quantum field theories using W states. *arXiv preprint arXiv:2505.03111*.